In [1]:
from MonteCarlo import MonteCarlo
import matplotlib.pyplot as plt
from scipy import constants as cst
import numpy as np

In [6]:
self = MonteCarlo(maximum_steps=10000,
    displace_mc=2,
    dumping_period=1000,
    thermo_period=1000,
    number_atoms=[5, 10],
    box_dimensions=[20, 20, 20],
    epsilon=[0.1, 0.1],
    sigma = [3, 6],
    atom_mass = [10, 10])
self.run()

step  N     T (K)     p (atm)   V (A3)    Ep (kcal/mol) Ek (kcal/mol) dens (g/cm3) 
0     15    3e+02     0.0       8e+03     1.69e+18      0.0           0.0311       
1000  15    3e+02     0.0       8e+03     3.63          0.0           0.0311       
2000  15    3e+02     0.0       8e+03     53.4          0.0           0.0311       
3000  15    3e+02     0.0       8e+03     2.86          0.0           0.0311       
4000  15    3e+02     0.0       8e+03     24.3          0.0           0.0311       
5000  15    3e+02     0.0       8e+03     36.7          0.0           0.0311       
6000  15    3e+02     0.0       8e+03     50.6          0.0           0.0311       
7000  15    3e+02     0.0       8e+03     7.78          0.0           0.0311       
8000  15    3e+02     0.0       8e+03     20.1          0.0           0.0311       
9000  15    3e+02     0.0       8e+03     30.0          0.0           0.0311       
10000 15    3e+02     0.0       8e+03     42.1          0.0           0.0311

In [6]:
cst.centi/cst.angstrom

100000000.0

In [ ]:
import sys, os, git, time

import MDAnalysis as mda
from MDAnalysis.analysis import distances

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo
from MolecularDynamics import MolecularDynamics

import numpy as np

In [ ]:
ti = time.time()

me = MinimizeEnergy(maximum_steps=10,
    number_atoms=[50],
    epsilon=[0.1],
    sigma = [3],
    atom_mass = [1],
    box_dimensions=[20, 20, 20],
    neighbor=10,
    thermo = 2,
    cut_off = 9,
    dump = 2)
me.perform_energy_minimization()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

In [ ]:
ti = time.time()

self = MolecularDynamics(maximum_steps = 10000,
    number_atoms = [20],
    epsilon = [0.1],
    sigma = [3],
    atom_mass = [1],
    tau_temp=100,
    box_dimensions=[20, 20, 20],
    neighbor = 10,
    thermo = 50,
    cut_off = 9,
    dump = 50)
self.run()

tf = time.time()

print("time =", np.round(tf-ti,2), "s")

In [ ]:
np.shape(self.evaluate_rij_matrix())

In [ ]:
r = np.linalg.norm(self.evaluate_rij_matrix(), axis=2)

In [ ]:
help(np.histogram)